In [164]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split

# Check if CUDA is available and set device to GPU if it is
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1337)
torch.cuda.manual_seed(1337)

In [165]:
class mlp(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_dim):
        super(mlp, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, hidden_dim3),
            nn.ReLU(),
            nn.Linear(hidden_dim3, output_dim)
            #nn.ReLU()
        )

    def forward(self, x):
        return self.network(x)

In [187]:
class rnn(nn.Module):
    def __init__(self, input_len, hidden_dim, num_layers=1):
        super(rnn, self).__init__()
        self.rnn_cell = nn.RNN(1, hidden_dim, num_layers, nonlinearity='relu', batch_first=True, bidirectional=True)
        self.rnns = nn.ModuleList([self.rnn_cell,
                   self.rnn_cell,
                   self.rnn_cell,
                   self.rnn_cell,
                   self.rnn_cell,
                   self.rnn_cell])
        self.fcs = nn.ModuleList([nn.Linear(24*2*hidden_dim, 10),
                   nn.Linear(10*2*hidden_dim, 9),
                   nn.Linear(9*2*hidden_dim, 8),
                   nn.Linear(8*2*hidden_dim, 7),
                   nn.Linear(7*2*hidden_dim, 6),
                   nn.Linear(6*2*hidden_dim, 5)])

    def forward(self, x):
        for i in range(6):
            x = torch.unsqueeze(x,dim=2)
            x, _ = self.rnns[i](x)
            x = torch.flatten(x,start_dim=1)
            x = self.fcs[i](x)
            if i==0:
                y = x
            else:
                y = torch.cat([y,x],dim=1)
        return y

In [188]:
class TrxEncoder(nn.Module):
    def __init__(self, d_token, n_head, dim_ffn):
        super(TrxEncoder, self).__init__()
        self.encoderLayer = nn.TransformerEncoderLayer(d_token, n_head, dim_ffn, batch_first=True)
        self.encoder = nn.TransformerEncoder(self.encoderLayer, num_layers=8, enable_nested_tensor=False)
        self.ffn = nn.Linear(24,10)
    
    def forward(self, x):
        x = x.reshape(len(x),24,1)
        y = self.encoder(x).squeeze()
        y = self.ffn(y)
        return y

In [189]:
class TR(nn.Module):
    def __init__(self):
        super().__init__()
        self.trx = nn.Transformer(d_model=1, nhead=1, num_encoder_layers=3,
                                           num_decoder_layers=3, dim_feedforward=32,
                                          batch_first=True)
        self.ffn = nn.Linear(24,10)
        
    def forward(self,x):
        x = x.reshape(len(x),24,1)
        y = self.trx(x).squeeze()
        y = self.ffn(y)
        return y

In [190]:
class dataset(Dataset):
    def __init__(self, volts_file, field_file):
        dfVolts = pd.read_csv(volts_file,header=None)
        dfField = pd.read_csv(field_file,header=None)
        N = len(dfVolts)
        self.input = torch.tensor(dfVolts.values).float()
        self.output = -torch.tensor(dfField.values).float()
    
    def __len__(self):
        return len(self.output)

    def __getitem__(self, idx):
        return self.input[idx], self.output[idx]

In [191]:
def train_test_split(dataset, train_percent):
    total_size = len(dataset)
    train_size = int(train_percent * total_size)  # for training
    test_size = total_size - train_size  # for testing
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    return train_dataset, test_dataset

In [192]:
def train(data_loader, model, optimizer, epoch, criterion, device):
    # set the model to training mode
    model.train()
    
    for (step, value) in enumerate(data_loader):
        data = value[0].to(device)
        target = value[1].to(device)
        optimizer.zero_grad()
        pred = model(data)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        
    return loss.item()

In [193]:
def test(data_loader, model, criterion, device):
    # set the model to evaluation mode
    model.eval()
    
    for (step, value) in enumerate(data_loader):
        data = value[0].to(device)
        target = value[1].to(device)
        pred = model(data)
        loss = criterion(pred, target)
        
    return loss.item()

In [194]:
volts_file = "SmartSurf_volts.csv"
field_file = "field_data.csv"

complete_set = dataset(volts_file, field_file)
train_set, test_set = train_test_split(complete_set, 0.8)

# setup hyperparameters
input_len = len(train_set[0][0])
output_dim = len(test_set[0][1])
hidden_dim = 3
N = train_set.__len__()

batch_size = 512
lr = 0.01
n_epoch = 1000

train_loader = DataLoader(
                train_set,
                batch_size=batch_size
            )
test_loader = DataLoader(
                test_set,
                batch_size=batch_size
            )

model = rnn(input_len, hidden_dim)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)   # Use Adam optimizer
criterion = nn.MSELoss()

# training and evaluation
for ep in range(n_epoch+1):
    train_loss = train(train_loader, model, optimizer, n_epoch, criterion, device)
    if ep%100 == 0:
        test_loss = test(test_loader, model, criterion, device)
        print(f"Epoch {ep}, train_loss: {train_loss}, test_loss: {test_loss}")

Epoch 0, train_loss: 740.3624267578125, test_loss: 549.4776611328125
Epoch 100, train_loss: 9.161927223205566, test_loss: 7.956477642059326
Epoch 200, train_loss: 8.409307479858398, test_loss: 7.447378158569336
Epoch 300, train_loss: 8.189200401306152, test_loss: 7.010048866271973
Epoch 400, train_loss: 5.378792762756348, test_loss: 5.1641459465026855
Epoch 500, train_loss: 4.830044746398926, test_loss: 4.644160747528076
Epoch 600, train_loss: 4.605642318725586, test_loss: 4.461787700653076
Epoch 700, train_loss: 4.358635902404785, test_loss: 4.201446533203125
Epoch 800, train_loss: 4.272604942321777, test_loss: 4.516939163208008
Epoch 900, train_loss: 4.0988688468933105, test_loss: 4.0090250968933105
Epoch 1000, train_loss: 4.1884284019470215, test_loss: 4.010580062866211


In [195]:
x,y = test_set[:]
x = x.to(device)
y = y.to(device)
print(x)

tensor([[ 2.1127,  0.4021,  0.0000,  ..., 16.2927, 14.4266, 14.8976],
        [10.4162, 12.6294, 12.2816,  ...,  2.3680,  0.1372,  0.0000],
        [ 9.7695, 11.4598, 13.3302,  ...,  2.9281,  4.4716,  7.4629],
        ...,
        [17.6748, 17.9147, 18.5436,  ..., 17.3052, 18.0118, 15.7994],
        [16.2400, 17.3081, 16.6672,  ..., 10.6252,  9.2832,  6.4089],
        [13.3738, 10.7777, 13.4109,  ...,  0.0231,  2.4387,  1.4984]],
       device='cuda:0')


In [196]:
pred = model(x)
print(f"Shape: {pred.shape}")
print(pred[0:10])

Shape: torch.Size([2000, 45])
tensor([[57.9640, 54.9392, 65.0804, 47.3603, 44.8649, 60.3948, 39.7857, 55.0832,
         42.5245, 51.6428, 53.1628, 52.0066, 38.0505, 45.1121, 55.5941, 55.9507,
         37.9351, 39.3011, 55.3061, 42.5513, 65.9100, 39.2098, 46.5347, 38.7779,
         43.1893, 39.1694, 52.3350, 63.3648, 52.2742, 43.9540, 57.2161, 44.1460,
         38.5167, 42.3642, 47.0849, 46.4182, 42.2051, 46.4139, 69.6231, 43.9085,
         53.1838, 64.9458, 52.9408, 42.4649, 56.0424],
        [50.4207, 56.6660, 58.7462, 43.6765, 43.6724, 59.2244, 39.1411, 47.6255,
         38.8712, 46.1347, 48.5837, 52.7150, 37.0502, 42.1403, 43.8487, 54.6295,
         34.4345, 39.1422, 54.8612, 38.0404, 64.1583, 33.6966, 47.6941, 35.4359,
         41.1460, 37.3913, 45.1290, 56.0497, 41.9138, 43.3193, 44.8648, 40.2711,
         37.2678, 40.5923, 48.3729, 42.0524, 39.5654, 46.0396, 55.6086, 42.4605,
         44.4678, 48.1048, 49.0735, 38.7525, 51.4980],
        [56.2372, 57.9833, 56.0176, 50.5492, 42.79

In [197]:
from sklearn import metrics

# Measure RMSE error.  RMSE is common for regression.

score = np.sqrt(metrics.mean_squared_error(pred.cpu().detach(), y.cpu().detach()))
print(f"Final score (RMSE): {score}")

score = torch.sqrt(torch.nn.functional.mse_loss(pred, y))
print(f"Final score (RMSE) with testing dataset: {score}")

Final score (RMSE): 2.0096938610076904
Final score (RMSE) with testing dataset: 2.0096938610076904


In [198]:
# Sample predictions
for i in range(50):
    print(f"{i+1}. Actual: {y[i]}, " + f"Predicted: {pred[i]}")

1. Actual: tensor([55.0596, 53.9631, 61.0375, 43.1159, 42.7885, 57.4072, 41.1819, 53.0643,
        44.0928, 53.9893, 52.9684, 50.1044, 37.7102, 45.7933, 52.5406, 55.2521,
        37.4063, 41.2118, 57.8127, 41.6962, 64.8062, 37.4063, 47.5780, 38.4484,
        42.2385, 40.2190, 52.0095, 58.6702, 51.1954, 44.8193, 55.7329, 43.4918,
        39.5647, 42.5225, 48.8466, 47.9462, 41.3588, 44.1824, 60.9985, 44.7397,
        51.3993, 61.8904, 52.5163, 43.2026, 56.9756], device='cuda:0'), Predicted: tensor([57.9640, 54.9392, 65.0804, 47.3603, 44.8649, 60.3948, 39.7857, 55.0832,
        42.5245, 51.6428, 53.1628, 52.0066, 38.0505, 45.1121, 55.5941, 55.9507,
        37.9351, 39.3011, 55.3061, 42.5513, 65.9100, 39.2098, 46.5347, 38.7779,
        43.1893, 39.1694, 52.3350, 63.3648, 52.2742, 43.9540, 57.2161, 44.1460,
        38.5167, 42.3642, 47.0849, 46.4182, 42.2051, 46.4139, 69.6231, 43.9085,
        53.1838, 64.9458, 52.9408, 42.4649, 56.0424], device='cuda:0',
       grad_fn=<SelectBackward0>)
2

49. Actual: tensor([48.9467, 57.8369, 58.6839, 43.4189, 40.7477, 59.4769, 37.5447, 45.4278,
        39.1696, 48.0378, 49.2204, 50.7103, 36.5026, 42.9787, 44.0178, 54.7335,
        34.3705, 38.5224, 56.8055, 38.6436, 64.0680, 34.3705, 47.0357, 35.9634,
        41.0511, 37.7198, 45.7661, 58.4310, 43.2980, 43.5321, 46.3232, 40.7126,
        37.5580, 40.2094, 48.0456, 42.3587, 39.8611, 46.4897, 57.9058, 43.1721,
        44.9180, 49.5392, 49.4113, 39.0710, 51.5156], device='cuda:0'), Predicted: tensor([51.3906, 57.1945, 59.7387, 44.9330, 42.7788, 59.0965, 38.2613, 47.7278,
        40.5439, 49.1663, 49.5203, 52.1707, 37.3798, 43.1484, 45.9141, 54.9532,
        35.1129, 38.9334, 57.3373, 39.4559, 64.0577, 34.8367, 48.2838, 36.4521,
        42.0685, 37.8067, 46.9480, 58.5560, 43.7963, 44.3168, 46.1608, 41.3471,
        37.9063, 41.2779, 49.4660, 42.9638, 40.6338, 46.6847, 57.2619, 43.3071,
        46.6595, 50.2099, 50.0931, 39.8220, 52.1534], device='cuda:0',
       grad_fn=<SelectBackward0>)


In [214]:
sampleIndex = 123
df_field = pd.read_csv("field_data.csv")
df_volts = pd.read_csv("SmartSurf_volts.csv")
# Pandas to Numpy
xt = df_volts.iloc[sampleIndex].values
yt = df_field.iloc[sampleIndex].values  # regression

# Numpy to PyTorch
xt = torch.tensor(xt, dtype=torch.float32).to(device)
xt = xt.reshape([1,24])
yt = torch.tensor(yt, dtype=torch.float32)

# Pass into model
yt_pred = model(xt).cpu().reshape([45,1])
print(yt_pred)

tensor([[52.3178],
        [57.4113],
        [59.7853],
        [44.3832],
        [43.8163],
        [59.2254],
        [39.2686],
        [48.0738],
        [39.9298],
        [47.1188],
        [48.8575],
        [53.3655],
        [36.9558],
        [42.7185],
        [44.6807],
        [55.1474],
        [34.5929],
        [39.0916],
        [55.9721],
        [38.6689],
        [64.1917],
        [34.0122],
        [48.0911],
        [35.7398],
        [41.4486],
        [37.5385],
        [45.4492],
        [56.7517],
        [42.5004],
        [43.8305],
        [45.1056],
        [40.6711],
        [37.5811],
        [40.8538],
        [48.9008],
        [42.4016],
        [39.9640],
        [46.2293],
        [55.8357],
        [42.7490],
        [45.1569],
        [48.5730],
        [49.5362],
        [39.2426],
        [51.5477]], grad_fn=<ViewBackward0>)


In [215]:
import csv
with open('pred.txt', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(yt_pred)

In [ ]:
y = rnn(x)
print(y)

In [78]:
rnn_1 = [nn.RNN(1,2,1),nn.RNN(2,2,1)]

In [178]:
x = torch.rand([2,24]).to(device)
model = nn.Linear(24, 10)
model = model.to(device)
y = model(x)